<h1> Compare - HaloPSA vs N-Able vs DattoRMM Inventory </h1>

Review for Asset Ingestion into HaloPSA Gaps or Issues

# Import Modules and Define Globals

In [ ]:
# data import and file manipulation
import requests
from requests.structures import CaseInsensitiveDict
import json
import csv
import xlrd

#data conditioning
import pandas as pd
import numpy as np
import re
import datetime as dt

#data visualization
[REDACTED]/.pyplot as plt
import seaborn as sns

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# dictionary Directory
dictionary_dir = 'd:/git/example_infrastructure_data_dev/dictionaries'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

## Create env Variables

In [ ]:
# import configparser for env secrets
from configparser import ConfigParser

config = ConfigParser()
config.read(f'{git_folder}/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

In [ ]:
# import and assign secrets from env.ini

dattormm_config = config['dattormm']
halopsa_config = config['halopsa']
auvik_config = config['auvik']

# Create Datto RMM DataFrame

## Create auth token

In [ ]:
# call token api url
token_uri = f"{dattormm_config['base_uri']}/auth/oauth/token"


# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'

# construct req body
data = CaseInsensitiveDict()
data['grant_type'] = 'password'
data['username'] = dattormm_config['api_key']
data['password'] = dattormm_config['api_secret']

# request content response
resp = requests.post(token_uri, headers=headers, data=data, auth=('public-client', 'public'))
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

access_token = c_dict['access_token']

## Create DataFrame via API Call Iteration


In [ ]:

## Create Devices DataFrame
# request content response
request_url = f"{dattormm_config['base_uri']}/api/v2/account/devices"

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)


# iterate and combine remaining pages
df_dattormm = pd.DataFrame(c_dict['devices'])
while c_dict['pageDetails']['nextPageUrl']:
    next_page = c_dict['pageDetails']['nextPageUrl']
    resp = requests.get(next_page, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)

    df_current_page = pd.DataFrame(c_dict['devices'])
    df_dattormm = pd.concat([df_dattormm, df_current_page], ignore_index=False)

# Create HaloPSA DataFrame


## Create Auth Token

In [ ]:
# call token api url
token_uri = f"{halopsa_config['base_uri']}/auth/token?tenant=example"

# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'


# construct req body
data = CaseInsensitiveDict()
data['grant_type'] = 'client_credentials'
data['client_id'] = halopsa_config['client_id']
data['client_secret'] = halopsa_config['client_secret']
data['scope'] = 'all'

# request content response
resp = requests.post(token_uri, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

access_token = c_dict['access_token']

In [ ]:
# request content response
request_url = f"{halopsa_config['base_uri']}/api/Client"

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# parameters req body
params = CaseInsensitiveDict()
params['count'] = 500

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, params=params)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)


# iterate and combine remaining pages
df_halopsa_clients = pd.DataFrame(c_dict['clients'])
try:
    while c_dict['pageDetails']['nextPageUrl']:
        next_page = c_dict['pageDetails']['nextPageUrl']
        resp = requests.get(next_page, headers=headers, data=data)
        content = resp.content.decode('utf-8')
        c_dict = json.loads(content)

        df_current_page = pd.DataFrame(c_dict['assets'])
        df_halopsa_clients = pd.concat([df_halopsa_clients, df_current_page], ignore_index=False)
except:
    print(f'All clients on first page.  Total Clients: {c_dict["record_count"]}')

## Create Asset DataFrame

In [ ]:
# request content response
request_url = f"{halopsa_config['base_uri']}/api/Asset"

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)


# iterate and combine remaining pages
df_halopsa = pd.DataFrame(c_dict['assets'])
try:
    while c_dict['pageDetails']['nextPageUrl']:
        next_page = c_dict['pageDetails']['nextPageUrl']
        resp = requests.get(next_page, headers=headers, data=data)
        content = resp.content.decode('utf-8')
        c_dict = json.loads(content)

        df_current_page = pd.DataFrame(c_dict['assets'])
        df_halopsa = pd.concat([df_halopsa, df_current_page], ignore_index=False)
except:
    print(f'All assets on first page.  Total Assets: {c_dict["record_count"]}')

# Import N-Able Devices CSV

In [ ]:
df_nable = pd.read_csv('[REDACTED]/.csv')

# Compare Halo to N-Able

In [ ]:
nable_clients = set(list(df_nable['clientName'].unique()))

In [ ]:
halopsa_asset_clients = set(list(df_halopsa['client_name'].unique()))

In [ ]:
halopsa_all_clients = set(list(df_halopsa_clients['name'].unique()))

In [ ]:
datto_clients = set(list(df_dattormm['siteName'].unique()))

# Differences between systems

## DattoRMM not in HaloPSA

In [ ]:
halopsa_asset_clients.symmetric_difference(datto_clients)

## N-Able not in HaloPSA

In [ ]:
remove_set = (halopsa_asset_clients.symmetric_difference(datto_clients)) #.symmetric_difference(nable_clients)

In [ ]:
nable_clients.intersection(halopsa_all_clients)

In [ ]:
string = str(nable_clients.intersection(halopsa_all_clients))

In [ ]:
string.replace('}','').replace('{','').replace("'",'')

In [ ]:
string_n